In [3]:
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np

In [4]:
mylist = []
for chunk in pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv', sep=',', chunksize=20000,low_memory=False):
    mylist.append(chunk)
p18 = pd.concat(mylist, axis = 0)
del mylist

In [5]:
#tackles
#tacklescoring - https://imgur.com/Ig0eCsh
#tacklepositions - LB, S, QB, RB, WR, TE, PK, PN
#groupby players and their tackles, create column to determine points from tackles, join with roster data, filter by eligible positions, 
tacklepts18one = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_1_player_name"] ).size()}).reset_index()
tacklepts18two = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_2_player_name"] ).size()}).reset_index()

In [6]:
#gotta figure out how to merge them, something about the column names not being indexed properly
tacklepts18 = pd.merge(tacklepts18one, tacklepts18two, left_on = 'assist_tackle_1_player_name', right_on = 'assist_tackle_2_player_name', how='inner')

In [7]:
#Add both groups of tackle assists together, multiply by 0.5 for fantasy point total
tacklepts18['tackles_xy'] = (tacklepts18['tackles_x'] + tacklepts18['tackles_y']) * 0.5
tacklepts18.head()

,assist_tackle_1_player_name,tackles_x,assist_tackle_2_player_name,tackles_y,tackles_xy
0,A.Adams,1,A.Adams,9,5.0
1,A.Amos,8,A.Amos,10,9.0
2,A.Anzalone,10,A.Anzalone,10,10.0
3,A.Armstead,10,A.Armstead,9,9.5
4,A.Bailey,11,A.Bailey,3,7.0


In [13]:
print(len(tacklepts18))

711


In [8]:
rosters18 = pd.read_csv(r'nflscrapR-data\roster_data\regular_season\reg_roster_2018.csv', sep = ',', low_memory = False)

In [9]:
print(len(rosters18))

544


In [10]:
#Add the column of  tackle assist points to the roster data
#Get rid of extra player id column
rosters18 = pd.merge(rosters18,tacklepts18[['tackles_xy','assist_tackle_1_player_name']],left_on='abbr_player_name',right_on='assist_tackle_1_player_name')
rosters18.head()

,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackles_xy,assist_tackle_1_player_name
0,2018,reg,Aaron Jones,A.Jones,GB,RB,00-0033293,6.5,A.Jones
1,2018,reg,Andy Jones,A.Jones,DET,WR,00-0032707,6.5,A.Jones
2,2018,reg,Aldrick Robinson,A.Robinson,MIN,WR,00-0028116,9.5,A.Robinson
3,2018,reg,Allen Robinson,A.Robinson,CHI,WR,00-0031428,9.5,A.Robinson
4,2018,reg,Alex Smith,A.Smith,WAS,QB,00-0023436,1.0,A.Smith


In [11]:
rosters18.drop(['assist_tackle_1_player_name'], axis=1,inplace=True)

In [12]:
rosters18.head()
print(len(rosters18))

80


In [12]:
qbhit18one = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_1_player_name"] ).size()}).reset_index()
qbhit18two = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_2_player_name"] ).size()}).reset_index()

In [13]:
qbhitpts18 = pd.merge(qbhit18one, qbhit18two, left_on = 'qb_hit_1_player_name', right_on = 'qb_hit_2_player_name', how='inner')

In [14]:
qbhitpts18['qb_hit_xy'] = qbhitpts18['qb_hit_x'] + qbhitpts18['qb_hit_y']
qbhitpts18.head()

,qb_hit_1_player_name,qb_hit_x,qb_hit_2_player_name,qb_hit_y,qb_hit_xy
0,A.Billings,6,A.Billings,1,7
1,A.Clayborn,12,A.Clayborn,1,13
2,A.Donald,40,A.Donald,1,41
3,A.Woods,6,A.Woods,1,7
4,B.Baker,1,B.Baker,2,3


In [15]:
rosters18.head()
print(len(rosters18))

,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackles_xy
0,2018,reg,Aaron Jones,A.Jones,GB,RB,00-0033293,6.5
1,2018,reg,Andy Jones,A.Jones,DET,WR,00-0032707,6.5
2,2018,reg,Aldrick Robinson,A.Robinson,MIN,WR,00-0028116,9.5
3,2018,reg,Allen Robinson,A.Robinson,CHI,WR,00-0031428,9.5
4,2018,reg,Alex Smith,A.Smith,WAS,QB,00-0023436,1.0


In [16]:
rosters18 = pd.merge(rosters18,qbhitpts18[['qb_hit_xy','qb_hit_1_player_name']],left_on='abbr_player_name',right_on='qb_hit_1_player_name')
rosters18.head()

,season,season_type,full_player_name,abbr_player_name,team,position,gsis_id,tackles_xy,qb_hit_xy,qb_hit_1_player_name
0,2018,reg,Brian Hill,B.Hill,ATL,RB,00-0033567,12.5,8,B.Hill
1,2018,reg,David Morgan,D.Morgan,MIN,TE,00-0032430,5.5,4,D.Morgan
2,2018,reg,Jordan Reed,J.Reed,WAS,TE,00-0030472,9.0,24,J.Reed
3,2018,reg,Josh Johnson,J.Johnson,WAS,QB,00-0026300,30.5,4,J.Johnson
4,2018,reg,Mark Ingram,M.Ingram,NO,RB,00-0027966,8.5,16,M.Ingram


In [17]:
print(len(rosters18))

5
